In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [3]:
#Load the csv exported in Part I to a DataFrame
cities = pd.read_csv("../WeatherPy/weatherdata.csv")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,Necochea,96,AR,1613317472,66,-38.5473,-58.7368,74.77,7.25
1,1,Chokurdakh,100,RU,1613317463,86,70.6333,147.9167,-11.96,9.60
2,2,New Norfolk,100,AU,1613317472,65,-42.7826,147.0587,61.00,1.99
3,3,Tyrma,8,RU,1613317476,88,50.0833,132.1667,-11.97,2.06
4,4,Busselton,95,AU,1613317477,85,-33.6500,115.3333,72.00,7.70


In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Humidity Heatmap
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

#Add Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions
#Drop any rows will null values

narrowed_city_df = cities.loc[(cities["Max Temperature"] > 70) & (cities["Max Temperature"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,50,São Filipe,0,CV,1613317529,67,14.8961,-24.4956,73.18,12.17
1,65,Pisco,0,PE,1613317546,78,-13.7000,-76.2167,73.40,9.22
2,73,Sungai Raya,0,ID,1613317555,88,0.7000,108.9000,75.56,9.48
3,112,Salalah,0,OM,1613317600,69,17.0151,54.0924,77.00,5.75
4,138,Lima,0,PE,1613317593,69,-12.0432,-77.0282,78.01,10.36


In [6]:
#Store into variable named hotel_df
#Add a "Hotel Name" column to the DataFrame
#Set parameters to search for hotels with 5000 meters
#Hit the Google Places API for each city's coordinates
#Store the first Hotel result into the DataFrame
#Plot markers on top of the heatmap

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,50,São Filipe,0,CV,1613317529,67,14.8961,-24.4956,73.18,12.17,Tortuga B&B
1,65,Pisco,0,PE,1613317546,78,-13.7000,-76.2167,73.40,9.22,Hotel San Isidro Oficial
2,73,Sungai Raya,0,ID,1613317555,88,0.7000,108.9000,75.56,9.48,Mess P.Milu
3,112,Salalah,0,OM,1613317600,69,17.0151,54.0924,77.00,5.75,HAMDAN PLAZA HOTEL SALALAH
4,138,Lima,0,PE,1613317593,69,-12.0432,-77.0282,78.01,10.36,Gran Hotel Bolivar Lima


In [9]:
# NOTE: Do not change any code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]


In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))